In [1]:
from obspy.core import Stream, UTCDateTime
from obspy.clients.fdsn import Client
from obspy.taup import TauPyModel
from obspy.geodetics import degrees2kilometers, kilometer2degrees, gps2dist_azimuth
import numpy as np
import pandas as pd
import os.path
import os

In [2]:
custom_model = TauPyModel(model =  '/Users/sydneydybing/SW4/strain/structure/mojave_noheader.npz')

In [3]:
path = '/Users/sydneydybing/StrainProject/2024/'

In [4]:
small_test = False

In [5]:
# Events

event_file = 'M6_catalog_2004-2024.csv'
events = pd.read_csv(path + event_file)
rows_for_extra_times = [8,9,14,16] # Events that need more identifying info because of duplicate dates
if small_test:
    events = events.iloc[[0,1,2]]
# events

In [6]:
# Stations

bsm_mda = pd.read_csv('/Users/sydneydybing/StrainProject/2024/bsm_metadata.csv')
stas = bsm_mda.loc[bsm_mda['LONG'] < -110] # North American stations only
if small_test:
    stas = stas.iloc[[0,1,2,3,4,5]]
# stas

In [7]:
init_array = np.zeros((len(events), len(stas)))
init_array.shape

(35, 76)

In [9]:
# Calculate theoretical P-wave arrival times for each event and each station in each event

for i in range(len(events)):
    
    ot = events.time.values[i]
    mag = events.mag.values[i]
    
    if i in rows_for_extra_times:
        cut_ot = ot[:13]
    else:
        cut_ot = ot[:10]
    
    print('Event ' + str(cut_ot) + 'M' + str(mag))
    
    hypo_lat = events.latitude.values[i]
    hypo_lon = events.longitude.values[i]
    hypo_depth = events.depth.values[i] # km
    
    for i2 in range(len(stas)):
        
        # sta_name = stas.BNUM.values[i2]
        # print(sta_name)
        
        sta_lat = stas.LAT.values[i2]
        sta_lon = stas.LONG.values[i2]
        dist_m, az, baz = gps2dist_azimuth(hypo_lat, hypo_lon, sta_lat, sta_lon) # dist in meters

        arrivals = custom_model.get_travel_times(source_depth_in_km = hypo_depth, 
                                      distance_in_degree = kilometer2degrees(dist_m/1000), phase_list=['p', 'P'])
        p_arr_time = arrivals[0].time
        # print(p_arr_time)

        # p_arr_times.append(p_arr_time)
        init_array[i,i2] = p_arr_time
    

Event 2023-04-13M6.0
Event 2022-12-20M6.4
Event 2022-11-22M6.2
Event 2021-12-20M6.2
Event 2021-07-08M6.0
Event 2020-05-15M6.5
Event 2020-03-31M6.5
Event 2019-12-25M6.3
Event 2019-12-23T20M6.0
Event 2019-12-23T19M6.0
Event 2019-08-29M6.3
Event 2019-07-06M7.1
Event 2019-07-04M6.4
Event 2019-07-04M6.2
Event 2018-10-22T06M6.5
Event 2018-10-22M6.8
Event 2018-10-22T05M6.5
Event 2018-08-22M6.2
Event 2016-12-08M6.6
Event 2014-08-24M6.02
Event 2014-04-24M6.5
Event 2014-03-10M6.8
Event 2013-09-04M6.0
Event 2013-09-03M6.1
Event 2012-12-14M6.3
Event 2012-11-08M6.1
Event 2012-04-11M6.0
Event 2011-09-09M6.4
Event 2010-04-04M7.2
Event 2010-01-10M6.5
Event 2008-01-10M6.3
Event 2005-06-17M6.6
Event 2005-06-15M7.2
Event 2004-11-02M6.7
Event 2004-07-19M6.4


In [10]:
init_array.shape

(35, 76)

In [11]:
np.save(path + 'theoretical_p_arrivals.npy', init_array)